# REST API set_metadata notebook

This notebook demonstrates set_metadata scoping for semantic sets using the MemMachine REST API.


## Prerequisites

- A MemMachine server running with semantic memory enabled.
- Set `MEMORY_BACKEND_URL` if the server is not at `http://127.0.0.1:8080`.


In [ ]:
import os
import time
import uuid
import requests

BASE_URL = os.environ.get("MEMORY_BACKEND_URL", "http://127.0.0.1:8080")
API_V2 = f"{BASE_URL}/api/v2"

def post(path, payload, expected_status=(200, 201, 204), timeout=30):
    resp = requests.post(f"{API_V2}{path}", json=payload, timeout=timeout)
    if resp.status_code not in expected_status:
        raise RuntimeError(f"{path} failed: {resp.status_code} {resp.text}")
    return resp

print(f"Using API base: {API_V2}")


In [ ]:
run_id = uuid.uuid4().hex[:8]
org_id = f"nb-org-{run_id}"
project_id = f"nb-project-{run_id}"

post(
    "/projects",
    {
        "org_id": org_id,
        "project_id": project_id,
        "description": "Notebook demo for set_metadata scoping",
    },
    expected_status=(201,),
)

print("Project created:", org_id, project_id)


In [ ]:
set_type_id = post(
    "/memories/semantic/set_type",
    {
        "org_id": org_id,
        "project_id": project_id,
        "is_org_level": False,
        "metadata_tags": ["user_id"],
        "name": "User Sets",
        "description": "User-scoped semantic sets",
    },
    expected_status=(201,),
).json()["set_type_id"]

print("Set type:", set_type_id)


In [ ]:
set_metadata_a = {"user_id": "user-a"}
set_metadata_b = {"user_id": "user-b"}

set_id_a = post(
    "/memories/semantic/set_id/get",
    {
        "org_id": org_id,
        "project_id": project_id,
        "is_org_level": False,
        "metadata_tags": ["user_id"],
        "set_metadata": set_metadata_a,
    },
).json()["set_id"]

set_id_b = post(
    "/memories/semantic/set_id/get",
    {
        "org_id": org_id,
        "project_id": project_id,
        "is_org_level": False,
        "metadata_tags": ["user_id"],
        "set_metadata": set_metadata_b,
    },
).json()["set_id"]

print("Set IDs:", set_id_a, set_id_b)


In [ ]:
def create_category_and_tag(set_id, category_name, tag_name):
    category_id = post(
        "/memories/semantic/category",
        {
            "org_id": org_id,
            "project_id": project_id,
            "set_id": set_id,
            "category_name": category_name,
            "prompt": f"{category_name} category",
            "description": f"{category_name} category for notebook demo",
        },
        expected_status=(201,),
    ).json()["category_id"]

    post(
        "/memories/semantic/category/tag",
        {
            "org_id": org_id,
            "project_id": project_id,
            "category_id": category_id,
            "tag_name": tag_name,
            "tag_description": f"{tag_name} tag",
        },
        expected_status=(201,),
    )

    return category_id

category_a = create_category_and_tag(set_id_a, "profile", "facts")
category_b = create_category_and_tag(set_id_b, "profile", "facts")

print("Categories:", category_a, category_b)


In [ ]:
feature_a = post(
    "/memories/semantic/feature",
    {
        "org_id": org_id,
        "project_id": project_id,
        "set_id": set_id_a,
        "category_name": "profile",
        "tag": "facts",
        "feature": "favorite_color",
        "value": "blue",
    },
    expected_status=(201,),
).json()["feature_id"]

feature_b = post(
    "/memories/semantic/feature",
    {
        "org_id": org_id,
        "project_id": project_id,
        "set_id": set_id_b,
        "category_name": "profile",
        "tag": "facts",
        "feature": "favorite_color",
        "value": "green",
    },
    expected_status=(201,),
).json()["feature_id"]

print("Feature IDs:", feature_a, feature_b)


In [ ]:
search_a = post(
    "/memories/search",
    {
        "org_id": org_id,
        "project_id": project_id,
        "query": "favorite color",
        "top_k": 5,
        "types": ["semantic"],
        "set_metadata": set_metadata_a,
    },
).json()

semantic_a = search_a["content"].get("semantic_memory") or []
assert semantic_a, "No semantic results returned for set_metadata search"
assert any(item["value"] == "blue" for item in semantic_a)
assert all(item["set_id"] == set_id_a for item in semantic_a)

print("Search A results:", len(semantic_a))


In [ ]:
list_b = post(
    "/memories/list",
    {
        "org_id": org_id,
        "project_id": project_id,
        "type": "semantic",
        "page_size": 50,
        "page_num": 0,
        "set_metadata": set_metadata_b,
    },
).json()

semantic_b = list_b["content"].get("semantic_memory") or []
assert semantic_b, "No semantic results returned for set_metadata list"
assert any(item["value"] == "green" for item in semantic_b)
assert all(item["set_id"] == set_id_b for item in semantic_b)

print("List B results:", len(semantic_b))


## Cleanup

The final cell deletes the semantic features, set type, and project created above.


In [ ]:
post(
    "/memories/semantic/delete",
    {
        "org_id": org_id,
        "project_id": project_id,
        "semantic_ids": [feature_a, feature_b],
    },
    expected_status=(204,),
)

post(
    "/memories/semantic/set_type/delete",
    {
        "org_id": org_id,
        "project_id": project_id,
        "set_type_id": set_type_id,
    },
    expected_status=(204,),
)

post(
    "/projects/delete",
    {
        "org_id": org_id,
        "project_id": project_id,
    },
    expected_status=(204,),
)

print("Cleanup complete.")
